<a href="https://colab.research.google.com/github/itsjustdannyb/style-transfer/blob/main/neural_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms.v2 as T
from torchvision.io import read_image

import numpy as np
import matplotlib.pyplot as plt

##  get images

In [2]:
IMAGE_DIR = 'images'

# create directory
!mkdir {IMAGE_DIR}

# download images to the directory you just created
!wget -q -O ./images/cafe.jpg https://cdn.pixabay.com/photo/2018/07/14/15/27/cafe-3537801_1280.jpg
!wget -q -O ./images/swan.jpg https://cdn.pixabay.com/photo/2017/02/28/23/00/swan-2107052_1280.jpg
!wget -q -O ./images/tnj.jpg https://i.dawn.com/large/2019/10/5db6a03a4c7e3.jpg
!wget -q -O ./images/rudolph.jpg https://cdn.pixabay.com/photo/2015/09/22/12/21/rudolph-951494_1280.jpg
!wget -q -O ./images/dynamite.jpg https://cdn.pixabay.com/photo/2015/10/13/02/59/animals-985500_1280.jpg
!wget -q -O ./images/painting.jpg https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg

print("image files you can choose from: ")
!ls images

mkdir: cannot create directory ‘images’: File exists
image files you can choose from: 
cafe.jpg  dynamite.jpg	painting.jpg  rudolph.jpg  swan.jpg  tnj.jpg


In [3]:
# set default images
content_path = f'{IMAGE_DIR}/swan.jpg'
style_path = f'{IMAGE_DIR}/painting.jpg'

In [41]:
## display content image and style image
def load_image(img):
  max_dim = 512

  # read image
  img = read_image(img).float().unsqueeze(dim=0)
  shape = [idx for idx in img.shape[2:]]
  long_dim = max(shape)

  scale = max_dim / long_dim
  new_shape = shape * scale
  return shape, img.shape

load_image(content_path)

TypeError: can't multiply sequence by non-int of type 'float'

In [44]:
max_dim = 512
img = read_image(content_path).float().unsqueeze(dim=0)
shape = [idx for idx in img.shape[2:]]
long_dim = max(shape)

scale = max_dim / long_dim
new_shape = shape * scale
return shape, img.shape

TypeError: can't multiply sequence by non-int of type 'float'

In [40]:
shape = [idx for idx in shape]
shape

[732, 1280]

In [8]:

def load_img(path_to_img):
  '''loads an image as a tensor and scales it to 512 pixels'''
  max_dim = 512
  image = tf.io.read_file(path_to_img)
  image = tf.image.decode_jpeg(image)
  image = tf.image.convert_image_dtype(image, tf.float32)

  shape = tf.shape(image)[:-1]
  shape = tf.cast(tf.shape(image)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  image = tf.image.resize(image, new_shape)
  image = image[tf.newaxis, :]
  image = tf.image.convert_image_dtype(image, tf.uint8)

  return image

In [11]:
tf_image = load_img(content_path)

In [53]:
max_dim = 512
img = tf.io.read_file(content_path) # read in the image, output is cryptic
img = tf.image.decode_jpeg(img) # decode the image
img = tf.image.convert_image_dtype(img, tf.float32)



shape = tf.shape(img)[:-1] # get height and width
shape = tf.cast(tf.shape(img)[:-1], tf.float32) # convert height and width to float32
long_dim = max(shape)
scale = max_dim / long_dim

new_shape = tf.cast(shape * scale, tf.int32)

img = tf.image.resize(img, new_shape)
img = img[tf.newaxis, :]
img = tf.image.convert_image_dtype(img, tf.uint8)

In [54]:
img.shape

TensorShape([1, 292, 512, 3])